In [1]:
import requests
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date, timedelta

In [2]:
api_base_url = 'https://api.ithacanet.tzkt.io/'
#aggregatorFactoryAddress = 'KT19K8MP6bc2MMszPZb56yihuMY19zqMxsLS' # 29 april
#aggregatorFactoryAddress = 'KT1RFV3Pz7GmfnnXAFxtwYdx3EGmnqHSBR27' # 2 may
#aggregatorFactoryAddress = 'KT1EXvLRpB3bTPBKzpmza6QYKkm2azVaLZr5' # 3 may
#aggregatorFactoryAddress = 'KT1BKiVTPLnbieisakZioJH3cC2MtfgamekD' # KMS
#aggregatorFactoryAddress = 'KT19NWEPVjWHF92obTVHHy9ghS9NWN27ikg2' # KMS 16 may
aggregatorFactoryAddress = 'KT1FoLiyDaDLqH3kx19RUjpzcvvbsBqfhzVD'# KMW with lambdas

In [3]:
r = requests.get(api_base_url + 'v1/contracts/' + aggregatorFactoryAddress + '/storage')
storage = r.json()

storage['trackedAggregators']

trackedAggregators = list(map(lambda x:{'name': x['key']['string_0'] + '/' + x['key']['string_1'], 'address': x['value']}, storage['trackedAggregators']))

trackedAggregators

[{'name': 'USD/BTC', 'address': 'KT1S2vk8wX4Ku6Qppoxbr8rm5bqqMwWVoTBf'},
 {'name': 'USD/DOGE', 'address': 'KT1St4X64aaXzTnMS7WUE32hhFyvzMWSTeUX'},
 {'name': 'USD/XTZ', 'address': 'KT1UKjG8xeehit3RqPao5a8WZ4vzSgRVVXyP'}]

In [4]:
# Choose aggregator

# 0 = BTC
# 1 = DOGE
# 2 = XTZ

aggregator_choice = 0

aggregator_address = trackedAggregators[aggregator_choice]['address']

print('Using aggregator ' +  trackedAggregators[aggregator_choice]['name'] + ' at ' + aggregator_address)

Using aggregator USD/BTC at KT1S2vk8wX4Ku6Qppoxbr8rm5bqqMwWVoTBf


In [5]:
# Fetch storage history

storage_history = []
last_number_of_storage_fetched = None
max_storage_per_request=1000

while not last_number_of_storage_fetched or last_number_of_storage_fetched == max_storage_per_request:
    
    last_id = str(storage_history[-1]['id']) if len(storage_history) != 0 else None
    if last_id is not None:
        url = api_base_url + 'v1/contracts/' + aggregator_address + '/storage/history?limit=' + str(max_storage_per_request) + '&lastId=' + last_id
    else:
        url = api_base_url + 'v1/contracts/' + aggregator_address + '/storage/history?limit=' + str(max_storage_per_request)
    req = requests.get(url)
    new_storage_history = req.json()
    
    last_number_of_storage_fetched = len(new_storage_history)
    storage_history += new_storage_history

print('Fetched ' + str(len(storage_history)) + ' storages from history')

Fetched 1366 storages from history


In [6]:
# Fetch operation history

operations = []
last_number_of_operations_fetched = None
max_operations_per_request=1000

while not last_number_of_operations_fetched or last_number_of_operations_fetched == max_operations_per_request:
    
    last_id = str(operations[-1]['id']) if len(operations) != 0 else None
    if last_id is not None:
        url = api_base_url + 'v1/operations/transactions?status=applied&target=' + aggregator_address + '&limit=' + str(max_operations_per_request) + '&offset=' + str(len(operations))
    else:
        url = api_base_url + 'v1/operations/transactions?status=applied&target=' + aggregator_address + '&limit=' + str(max_operations_per_request)

    req = requests.get(url)
    new_operations_history = req.json()

    last_number_of_operations_fetched = len(new_operations_history)
    operations += new_operations_history

print('Fetched ' + str(len(operations)) + ' operations')

Fetched 1365 operations


In [7]:
# Create observations dataframe

decimals = int(storage_history[0]['value']['aggregatorConfig']['decimals'])

set_observation_operations = [op for op in operations if 'parameter' in op and op['parameter']['entrypoint'] in ['setObservationReveal']]


observations = [{
    'observation_level': int(op['level']),
    'hash': op['hash'],
    'observation_time': np.datetime64(op['timestamp']),
    'round': int(op['parameter']['value']['roundId']),
    'observation': int(op['parameter']['value']['priceSalted']['nat']) / 10 ** decimals,
    'oracle': op['sender']['address'],
    'cost': int(op['storageFee']) + int(op['bakerFee'])
 } for op in set_observation_operations]

df_observations = pd.DataFrame(data = observations)

df_observations.head()

KeyError: 'aggregatorConfig'

In [ ]:
# Fetch balance history

oracle_addresses =  df_observations['oracle'].unique().tolist()
intresting_addresses = df_observations['oracle'].unique().tolist()
intresting_addresses.append(aggregator_address)

balances_history = []

for address in intresting_addresses:
    address_balances = []
    last_number_of_balances_fetched = None
    max_balances_per_request=1000

    while not last_number_of_balances_fetched or last_number_of_balances_fetched == max_balances_per_request:

        last_id = str(operations[-1]['id']) if len(operations) != 0 else None
        if last_id is not None:
            url = api_base_url + 'v1/accounts/' + address + '/balance_history?limit=' + str(max_balances_per_request) + '&offset=' + str(len(address_balances))
        else:
            url = api_base_url + 'v1/accounts/' + address + '/balance_history?limit=' + str(max_balances_per_request)
        req = requests.get(url)
        new_address_balances = req.json()

        last_number_of_balances_fetched = len(new_address_balances)
        balances_history += [{**x, 'address': address} for x in new_address_balances]
        address_balances += [{**x, 'address': address} for x in new_address_balances]


In [ ]:
# Create request rate update (normal + deviation) dataframe

request_rate_update_storage = [x for x in storage_history if 'parameter' in x['operation'] and x['operation']['parameter']['entrypoint'] in ['requestRateUpdate', 'requestRateUpdateDeviation']]

start_of_round = [{
    'round': int(x['value']['round']),
    'start_level': x['level'],
    'start_time': np.datetime64(x['timestamp']),
    'start_hash': x['operation']['hash'],
    'previous_median': int(x['value']['lastCompletedRoundPrice']['price']) / 10 ** decimals,
    'time_round': np.datetime64(x['timestamp']),
    'deviation_round': x['operation']['parameter']['entrypoint'] == 'requestRateUpdateDeviation',
#    'start_reward_poll': sum(map(lambda y:int(y), x['value']['oracleRewardsXTZ'].values()))
 } for x in request_rate_update_storage]


end_of_round = [{
    'round': int(x['value']['round']) - 1,
    'end_level': x['level'],
    'end_time': np.datetime64(x['timestamp']),
    'end_hash': x['operation']['hash'],
    'end_reward': sum(map(lambda y:int(y), x['value']['oracleRewardsXTZ'].values())),
    'median': int(x['value']['lastCompletedRoundPrice']['price']) / 10 ** decimals if int(x['value']['round']) - 1 == int(x['value']['lastCompletedRoundPrice']['round']) else None,
    'completed': int(x['value']['round']) - 1 == int(x['value']['lastCompletedRoundPrice']['round'])
#    'end_reward_poll': sum(map(lambda y:int(y), x['value']['oracleRewardsXTZ'].values()))
 } for x in request_rate_update_storage]

#request_rate_update_storage

#df_request_rate_updates = pd.DataFrame(data = start_of_round).sort_values(by="previous_round")

#df_request_rate_updates['previous_round_deviation'] = df_request_rate_updates['previous_median'].diff() / df_request_rate_updates['previous_median'] * 100

#df_request_rate_updates
df_start_of_round = pd.DataFrame(data = start_of_round).sort_values(by="round")
df_end_of_round = pd.DataFrame(data = end_of_round).sort_values(by="round")

df_round = df_start_of_round.merge(df_end_of_round, how='inner', on='round')

df_round['deviation'] = 100 * (df_round['median'] - df_round['previous_median']) /  df_round['previous_median']
df_round.head()

In [ ]:
# Create balances dataframe
balances_history_tmp = [{
    'level': int(b['level']),
    'time': np.datetime64(b['timestamp']),
    'balance': b['balance'] / 1_000_000,
    'address': b['address'],
 } for b in balances_history]

df_balances = pd.DataFrame(data=balances_history_tmp)

df_balances.head()

In [ ]:
failed_deviations_rounds = df_round.loc[df_round['deviation_round'] & (df_round['deviation'].abs() < 0.1)]
failed_deviations_rounds.head()

In [ ]:
df_all = df_round.merge(df_observations, how='inner', on='round')
df_all.head()

In [ ]:
sns.set_theme(style="darkgrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
f, ax = plt.subplots(figsize=(25, 10))

sns.lineplot(
    data=df_all, 
    x="observation_time", 
    y="observation",
    color=".7", 
    hue="oracle", 
    linewidth=1, 
    ax=ax,
    style="oracle",
    markers=True,
    dashes=False
)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


sns.set_theme(style="darkgrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
f, ax = plt.subplots(figsize=(25, 10))

sns.lineplot(
    data=df_all[df_all['observation_time'] > (np.datetime64(date.today() - timedelta(days=1)))], 
    x="observation_time", 
    y="observation",
    color=".7", 
    hue="oracle", 
    linewidth=1, 
    ax=ax,
    style="oracle",
    markers=True,
    dashes=False
)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

#sns.set_theme(style="darkgrid")
#sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
#f, ax = plt.subplots(figsize=(25, 10))
#
#sns.lineplot(
#    data=df_all, 
#    x="round", 
#    y="observation",
#    color=".7", 
#    hue="oracle", 
#    linewidth=1, 
#    ax=ax,
#    style="oracle",
#    markers=True,
#    dashes=False
#)
#
#ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
df_all['oracle_deviation'] = 100 * (df_all['observation'] - df_all['median']) / df_all['median']
df_all['oracle_previous_median_deviation'] = 100 * (df_all['observation'] - df_all['previous_median']) / df_all['previous_median']

sns.set_theme(style="darkgrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
f, ax = plt.subplots(figsize=(12, 5))

scatterplot = sns.scatterplot(
    data=df_all, 
    x="round", 
    y="oracle_deviation",
    color=".7", 
    hue="oracle", 
    linewidth=1, 
    ax=ax,
    style="oracle",
    s=100
)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


sns.set_theme(style="darkgrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
f, ax = plt.subplots(figsize=(12, 5))

deviation_serie = df_all['oracle_deviation'].abs()


scatterplot = sns.histplot(
    data=df_all,
    x=deviation_serie[deviation_serie != 0],
    ax=ax,
    bins=20,
    kde=True
)


In [ ]:
sns.set_theme(style="darkgrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
f, ax = plt.subplots(figsize=(20, 15))

# ~df_all["deviation_round"] | (df_all["deviation_round"] & ~(df_all['deviation'].abs() < 0.5))

df_all['round_label'] = 'normal'
df_all.loc[df_all["deviation_round"] & ~(df_all['deviation'].abs() < 0.5), 'round_label'] = 'deviation'
df_all.loc[df_all["deviation_round"] & (df_all['deviation'].abs() < 0.5), 'round_label'] = 'failed deviation'

sns.scatterplot(
    data=df_all, 
    x="observation_time", 
    y=df_all['deviation'].abs(),
    color=".7", 
    linewidth=1, 
    ax=ax,
    hue="round_label", 
    s=100
)

ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
ax.plot()

In [ ]:
perthousandDeviationTrigger = int(storage_history[0]['value']['aggregatorConfig']['perthousandDeviationTrigger'])

failed_deviations_rounds = df_all.loc[df_all['deviation_round'] & (df_all['deviation'].abs() < 0.1 * perthousandDeviationTrigger/2)]

col_to_display = [
    'round',
    'observation_level',
    'oracle',
    'observation',
    'oracle_previous_median_deviation',
    'median'
]
failed_deviations_rounds[col_to_display].head()

In [ ]:
sns.set_theme(style="darkgrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
f, ax = plt.subplots(figsize=(15, 8))

ax.set_xlim(0, None)

sns.histplot(data=df_round, x=df_round[df_round['deviation_round']]['deviation'].abs(), bins=20,)

In [ ]:
observation_reward_xtz = int(storage_history[0]['value']['aggregatorConfig']['rewardAmountXTZ'])
deviation_reward_xtz = 0
aggregator_start = df_all['start_time'].min()
aggregator_end = df_all['end_time'].max()
uptime = aggregator_end - aggregator_start

uptime_days = uptime.total_seconds() / (60 * 60 * 24)

n_observations = len(df_observations)
n_observations_per_day = round(n_observations / uptime_days, 2)

n_rounds = len(df_round)
n_rounds_per_day = round(n_rounds / uptime_days, 2)

n_deviation_rounds = df_round['deviation_round'].sum()
n_deviation_rounds_per_day = round(n_deviation_rounds / uptime_days, 2)


n_non_deviation_rounds = n_rounds - n_deviation_rounds
n_non_deviation_rounds_per_day = round(n_non_deviation_rounds / uptime_days, 2)


total_emited_rewards_mutez = observation_reward_xtz * n_observations + deviation_reward_xtz * n_deviation_rounds
total_emited_rewards_xtz = total_emited_rewards_mutez / 1_000_000
total_emited_rewards_xtz_per_day = round(total_emited_rewards_xtz / uptime_days, 2)
total_emited_rewards_xtz_per_month = round((total_emited_rewards_xtz / uptime_days) * 365/12, 2)

print('Reward per observation: ' + str(observation_reward_xtz) + ' mutez')
print('Aggregator uptime: ' + str(uptime))
print('')
print('Number of observations: ' + str(n_observations) + ' total (' + str(n_observations_per_day) + ' /day)')
print('Number of rounds: ' + str(n_rounds) + ' total (' + str(n_rounds_per_day) + ' /day)' )
print('Number of deviation rounds: ' + str(n_deviation_rounds) + ' total (' + str(n_deviation_rounds_per_day) + ' /day)' )
print('Number of non deviation rounds: ' + str(n_non_deviation_rounds) + ' total (' + str(n_non_deviation_rounds_per_day) + ' /day)' )
print('')
print('Upkeep cost: ' + str(total_emited_rewards_xtz) + ' XTZ total (' + str(total_emited_rewards_xtz_per_day) + ' XTZ /day) (' + str(total_emited_rewards_xtz_per_month) + ' XTZ /month)'   )



In [ ]:
# Oracle balances

df_oracle_balance_only = df_balances[df_balances['address'].isin(oracle_addresses)]

aggregator_start = df_all['start_time'].min()
aggregator_end = df_all['end_time'].max()

sns.set_theme(style="darkgrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
f, ax = plt.subplots(figsize=(25, 10))


sns.lineplot(
    data=df_oracle_balance_only[(df_oracle_balance_only['time'] > aggregator_start) & (df_oracle_balance_only['time'] < aggregator_end)], 
    x="time", 
    y="balance",
    color=".7", 
    hue="address", 
    linewidth=1, 
    ax=ax,
    style="address",
    dashes=False
)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
# Oracle balances

df_oracle_balance_only = df_balances[~df_balances['address'].isin(oracle_addresses)]

sns.set_theme(style="darkgrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
f, ax = plt.subplots(figsize=(25, 10))

sns.lineplot(
    data=df_oracle_balance_only[(df_oracle_balance_only['time'] > aggregator_start) & (df_oracle_balance_only['time'] < aggregator_end)], 
    x="time", 
    y="balance",
    color=".7", 
    hue="address", 
    linewidth=1, 
    ax=ax,
    style="address",
    dashes=False,
    legend=False
)